In [1]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from utils import *
import resnet50
import pandas as pd
import numpy as np
from glob import glob
from keras.layers import Dense, Conv2D, BatchNormalization, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.metrics import log_loss
%matplotlib inline

data_dir = os.path.join(os.getcwd(), 'data', 'whale')
img_dir  = os.path.join(os.getcwd(), 'data', 'whale', 'crops')
batch_size=64
num_class = 447
img_shape=(3, 224, 224)
target_size = (224,224)

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [11]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import os

def create_sub(preds):
    train_batches = ImageDataGenerator().flow_from_directory(os.getcwd()+'/data/whale/crops/train', shuffle=False)
    test_batches = ImageDataGenerator().flow_from_directory(os.getcwd()+'/data/whale/crops/test', shuffle=False)
    file_names = np.array([f[f.find('/') +1:] for f in test_batches.filenames])
    
    idx_class = [(v, k) for k,v in train_batches.class_indices.items()]
    idx_class= sorted(idx_class, key=lambda x: x[0])
    classes = [x[1] for x in idx_class]
    data = np.hstack((file_names[:, np.newaxis], preds))
    columns = ['Image'] + classes
    return pd.DataFrame(data, columns=columns)

In [2]:
data_dir

'/home/ubuntu/courses/deeplearning1/nbs/data/whale'

In [3]:
img_dir

'/home/ubuntu/courses/deeplearning1/nbs/data/whale/crops'

In [22]:
ImageDataGenerator?

In [2]:
def get_tr_batches(dr = img_dir+ '/train', gen = ImageDataGenerator(), batch_size=batch_size):
    return gen.flow_from_directory(dr, shuffle=True, target_size=target_size, batch_size=64)

def get_val_batches(dr = img_dir+ '/valid', gen = ImageDataGenerator(), batch_size=batch_size):
    return gen.flow_from_directory(dr, shuffle=False, target_size=target_size, batch_size=batch_size)


def get_test_batches(dr = img_dir+ '/test', gen = ImageDataGenerator(), batch_size=batch_size):
    return gen.flow_from_directory(dr, shuffle=False, target_size=target_size, batch_size=batch_size)

In [6]:
def get_model():
    vgg = Vgg16BN()
    model = vgg.model
    last_conv = [i for i, v in enumerate(model.layers) if type(v) is Conv2D][-1]
    for layer in model.layers[: last_conv+1]: layer.trainable=False
    model.layers = model.layers[:-1]
    model.add(Dense(num_class, activation='softmax'))
    return model




In [ ]:
model = get_model()
augmenter = ImageDataGenerator(rotation_range=180, horizontal_flip=True, vertical_flip=True, zoom_range=0.10)
tr_batches = get_tr_batches(gen=augmenter)
val_b = get_val_batches()



In [7]:
model.compile(Adam(0.001), 'categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=10)

Epoch 1/10
4092/4092 [==============================] - 116s - loss: 6.0953 - acc: 0.0051 - val_loss: 6.0779 - val_acc: 0.0044
Epoch 2/10
4092/4092 [==============================] - 117s - loss: 6.0698 - acc: 0.0071 - val_loss: 6.0564 - val_acc: 0.0044
Epoch 3/10
4092/4092 [==============================] - 117s - loss: 6.0470 - acc: 0.0076 - val_loss: 6.0377 - val_acc: 0.0044
Epoch 4/10
4092/4092 [==============================] - 117s - loss: 6.0306 - acc: 0.0090 - val_loss: 6.0229 - val_acc: 0.0066
Epoch 5/10
4092/4092 [==============================] - 117s - loss: 6.0152 - acc: 0.0100 - val_loss: 6.0109 - val_acc: 0.0088
Epoch 6/10
4092/4092 [==============================] - 117s - loss: 5.9999 - acc: 0.0078 - val_loss: 5.9944 - val_acc: 0.0044
Epoch 7/10
4092/4092 [==============================] - 117s - loss: 5.9844 - acc: 0.0090 - val_loss: 5.9850 - val_acc: 0.0044
Epoch 8/10
4092/4092 [==============================] - 117s - loss: 5.9731 - acc: 0.0073 - val_loss: 5.9731 - 

In [10]:
test_b = get_test_batches()

Found 6925 images belonging to 1 classes.


In [12]:
model.predict_generator?

In [13]:
preds = model.predict_generator(test_b, test_b.nb_sample)

In [16]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import os

def create_sub(preds):
    train_batches = ImageDataGenerator().flow_from_directory(os.getcwd()+'/data/whale/crops/train', shuffle=False)
    test_batches = ImageDataGenerator().flow_from_directory(os.getcwd()+'/data/whale/crops/test', shuffle=False)
    file_names = np.array([f[f.find('/') +1:] for f in test_batches.filenames])
    
    idx_class = [(v, k) for k,v in train_batches.class_indices.items()]
    idx_class= sorted(idx_class, key=lambda x: x[0])
    classes = [x[1] for x in idx_class]
    data = np.hstack((file_names[:, np.newaxis], preds))
    columns = ['Image'] + classes
    return pd.DataFrame(data, columns=columns)

In [17]:

sub = create_sub(preds)


Found 4092 images belonging to 447 classes.
Found 6925 images belonging to 1 classes.


In [18]:
sub.head()

,Image,whale_00195,whale_00442,whale_02411,whale_02608,whale_02839,whale_03103,whale_03227,whale_03623,whale_03728,...,whale_98618,whale_98633,whale_98645,whale_98746,whale_98939,whale_98996,whale_99243,whale_99326,whale_99558,whale_99573
0,w_8037.jpg,0.00262654665858,0.00135857670102,0.00258631142788,0.00241854321212,0.00161823036615,0.00125579128508,0.00293743144721,0.00207300856709,0.00166204257403,...,0.00182293355465,0.00169122137595,0.00284156017005,0.00262740231119,0.00209450535476,0.0023715433199,0.00183011766057,0.00186924217269,0.00270950142294,0.00249393563718
1,w_4494.jpg,0.00275591900572,0.00184231193271,0.00239349715412,0.00209003686905,0.00175024918281,0.00156655488536,0.00264618988149,0.00197285437025,0.00187432509847,...,0.00224226340652,0.00233909208328,0.00266989110969,0.00293063628487,0.00204541953281,0.00222271936946,0.00218259682879,0.0019323034212,0.00266992556863,0.00227277912199
2,w_4673.jpg,0.00262266141362,0.00134813564364,0.00259037897922,0.00242638378404,0.00161464547273,0.00124867085833,0.00294397864491,0.00207485840656,0.00165667757392,...,0.00181321811397,0.00167741952464,0.00284498091787,0.00261962017976,0.00209505995736,0.00237454194576,0.00182173168287,0.00186715344898,0.00270962622017,0.00249879201874
3,w_8273.jpg,0.00262266141362,0.00134813564364,0.00259037897922,0.00242638378404,0.00161464547273,0.00124867085833,0.00294397864491,0.00207485840656,0.00165667757392,...,0.00181321811397,0.00167741952464,0.00284498091787,0.00261962017976,0.00209505995736,0.00237454194576,0.00182173168287,0.00186715344898,0.00270962622017,0.00249879201874
4,w_7611.jpg,0.00298971543089,0.00176843593363,0.00248428061604,0.00168860983104,0.00146184314508,0.00147105299402,0.0031328597106,0.00233552721329,0.0021035191603,...,0.00242493441328,0.00237485021353,0.00248693604954,0.00299931457266,0.00193240272347,0.00179956515785,0.00216462975368,0.00196502404287,0.00283952848986,0.00237279315479


In [19]:
sub.to_csv('crop_head_first.csv', index=False)

from IPython.display import FileLink
FileLink('crop_head_first.csv')


/home/ubuntu/courses/deeplearning1/nbs/crop_head_first.csv

scores 5.99413

# Train Longer

In [15]:
model = get_model()
augmenter = ImageDataGenerator(rotation_range=180, horizontal_flip=True, vertical_flip=True, zoom_range=0.10)
tr_batches = get_tr_batches(gen=augmenter)
val_b = get_val_batches()



Found 4092 images belonging to 447 classes.
Found 452 images belonging to 447 classes.


In [16]:
model.compile(Adam(0.001), 'categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=15)

Epoch 1/15
4092/4092 [==============================] - 122s - loss: 6.0961 - acc: 0.0039 - val_loss: 6.0805 - val_acc: 0.0088
Epoch 2/15
4092/4092 [==============================] - 124s - loss: 6.0736 - acc: 0.0086 - val_loss: 6.0590 - val_acc: 0.0088
Epoch 3/15
4092/4092 [==============================] - 124s - loss: 6.0548 - acc: 0.0095 - val_loss: 6.0439 - val_acc: 0.0111
Epoch 4/15
4092/4092 [==============================] - 124s - loss: 6.0376 - acc: 0.0095 - val_loss: 6.0281 - val_acc: 0.0088
Epoch 5/15
4092/4092 [==============================] - 124s - loss: 6.0197 - acc: 0.0100 - val_loss: 6.0032 - val_acc: 0.0088
Epoch 6/15
4092/4092 [==============================] - 124s - loss: 6.0036 - acc: 0.0095 - val_loss: 5.9894 - val_acc: 0.0111
Epoch 7/15
4092/4092 [==============================] - 124s - loss: 5.9886 - acc: 0.0100 - val_loss: 5.9790 - val_acc: 0.0111
Epoch 8/15
4092/4092 [==============================] - 125s - loss: 5.9754 - acc: 0.0125 - val_loss: 5.9650 - 

In [9]:
model.save_weights('crop_vgg_aug_5.9108.h5')

In [10]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=5)

Epoch 1/5
4092/4092 [==============================] - 123s - loss: 5.9001 - acc: 0.0147 - val_loss: 5.8965 - val_acc: 0.0199
Epoch 2/5
4092/4092 [==============================] - 124s - loss: 5.8951 - acc: 0.0144 - val_loss: 5.8900 - val_acc: 0.0155
Epoch 3/5
4092/4092 [==============================] - 125s - loss: 5.8894 - acc: 0.0130 - val_loss: 5.8879 - val_acc: 0.0155
Epoch 4/5
4092/4092 [==============================] - 125s - loss: 5.8865 - acc: 0.0152 - val_loss: 5.8896 - val_acc: 0.0155
Epoch 5/5
4092/4092 [==============================] - 125s - loss: 5.8763 - acc: 0.0134 - val_loss: 5.8904 - val_acc: 0.0155


In [11]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=5)

Epoch 1/5
4092/4092 [==============================] - 123s - loss: 5.8801 - acc: 0.0134 - val_loss: 5.8836 - val_acc: 0.0155
Epoch 2/5
4092/4092 [==============================] - 124s - loss: 5.8708 - acc: 0.0115 - val_loss: 5.8735 - val_acc: 0.0177
Epoch 3/5
4092/4092 [==============================] - 125s - loss: 5.8717 - acc: 0.0112 - val_loss: 5.8721 - val_acc: 0.0199
Epoch 4/5
4092/4092 [==============================] - 125s - loss: 5.8628 - acc: 0.0137 - val_loss: 5.8718 - val_acc: 0.0177
Epoch 5/5
4092/4092 [==============================] - 125s - loss: 5.8654 - acc: 0.0112 - val_loss: 5.8773 - val_acc: 0.0133


In [13]:
model.save_weights('crop_vgg_aug_5.8773.h5')

In [7]:
model.load_weights('crop_vgg_aug_5.8773.h5')

In [8]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=10)

Epoch 1/10
4092/4092 [==============================] - 122s - loss: 5.8556 - acc: 0.0132 - val_loss: 5.8617 - val_acc: 0.0155
Epoch 2/10
4092/4092 [==============================] - 123s - loss: 5.8457 - acc: 0.0132 - val_loss: 5.8499 - val_acc: 0.0155
Epoch 3/10
4092/4092 [==============================] - 123s - loss: 5.8424 - acc: 0.0144 - val_loss: 5.8533 - val_acc: 0.0133
Epoch 4/10
4092/4092 [==============================] - 123s - loss: 5.8310 - acc: 0.0144 - val_loss: 5.8431 - val_acc: 0.0199
Epoch 5/10
4092/4092 [==============================] - 123s - loss: 5.8254 - acc: 0.0159 - val_loss: 5.8432 - val_acc: 0.0221
Epoch 6/10
4092/4092 [==============================] - 124s - loss: 5.8263 - acc: 0.0144 - val_loss: 5.8423 - val_acc: 0.0199
Epoch 7/10
4092/4092 [==============================] - 124s - loss: 5.8225 - acc: 0.0149 - val_loss: 5.8407 - val_acc: 0.0199
Epoch 8/10
4092/4092 [==============================] - 124s - loss: 5.8183 - acc: 0.0154 - val_loss: 5.8454 - 

In [9]:
model.save_weights('crop_vgg_aug_5.8402.h5')

In [10]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=10)

Epoch 1/10
4092/4092 [==============================] - 122s - loss: 5.8127 - acc: 0.0149 - val_loss: 5.8423 - val_acc: 0.0199
Epoch 2/10
4092/4092 [==============================] - 123s - loss: 5.8080 - acc: 0.0120 - val_loss: 5.8356 - val_acc: 0.0199
Epoch 3/10
4092/4092 [==============================] - 124s - loss: 5.8109 - acc: 0.0147 - val_loss: 5.8396 - val_acc: 0.0177
Epoch 4/10
4092/4092 [==============================] - 124s - loss: 5.8039 - acc: 0.0142 - val_loss: 5.8345 - val_acc: 0.0177
Epoch 5/10
4092/4092 [==============================] - 124s - loss: 5.8045 - acc: 0.0130 - val_loss: 5.8310 - val_acc: 0.0177
Epoch 6/10
4092/4092 [==============================] - 124s - loss: 5.8107 - acc: 0.0144 - val_loss: 5.8287 - val_acc: 0.0177
Epoch 7/10
4092/4092 [==============================] - 124s - loss: 5.7991 - acc: 0.0159 - val_loss: 5.8368 - val_acc: 0.0133
Epoch 8/10
4092/4092 [==============================] - 124s - loss: 5.8015 - acc: 0.0134 - val_loss: 5.8415 - 

KeyboardInterrupt: 

In [11]:
model.save_weights('crop_vgg_aug_5.8299.h5')

In [17]:
model.load_weights('crop_vgg_aug_5.8299.h5')

In [9]:
test_b = get_test_batches()

Found 6925 images belonging to 1 classes.


In [10]:
preds = model.predict_generator(test_b, test_b.nb_sample)

In [12]:
sub = create_sub(preds)

Found 4092 images belonging to 447 classes.
Found 6925 images belonging to 1 classes.


In [14]:
sub.to_csv('crop_head_second.csv', index=False)

from IPython.display import FileLink
FileLink('crop_head_second.csv')


/home/ubuntu/courses/deeplearning1/nbs/crop_head_second.csv

scores 5.9379. That's 10 more than the validation scores. which makes me think that maybe you need better bounding boxes...

## Train more

In [19]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=20)

Epoch 1/20
4092/4092 [==============================] - 122s - loss: 5.7680 - acc: 0.0147 - val_loss: 5.8310 - val_acc: 0.0177
Epoch 2/20
4092/4092 [==============================] - 123s - loss: 5.7890 - acc: 0.0156 - val_loss: 5.8284 - val_acc: 0.0133
Epoch 3/20
4092/4092 [==============================] - 123s - loss: 5.7752 - acc: 0.0161 - val_loss: 5.8405 - val_acc: 0.0133
Epoch 4/20
4092/4092 [==============================] - 123s - loss: 5.7751 - acc: 0.0161 - val_loss: 5.8274 - val_acc: 0.0155
Epoch 5/20
4092/4092 [==============================] - 123s - loss: 5.7695 - acc: 0.0156 - val_loss: 5.8427 - val_acc: 0.0133
Epoch 6/20
4092/4092 [==============================] - 123s - loss: 5.7808 - acc: 0.0120 - val_loss: 5.8513 - val_acc: 0.0111
Epoch 7/20
4092/4092 [==============================] - 123s - loss: 5.7570 - acc: 0.0154 - val_loss: 5.8310 - val_acc: 0.0177
Epoch 8/20
4092/4092 [==============================] - 123s - loss: 5.7734 - acc: 0.0154 - val_loss: 5.8294 - 

KeyboardInterrupt: 

In [ ]:
#

The validation loss has started decreasing very slowly. This indicates that another architecture may perform better. Perhaps you should try the vgg architecture, without any pre-trained weights.

# Set up data for faster training and sampling

In [23]:
tr_b = ImageDataGenerator().flow_from_directory(img_dir+'/train',
                                                shuffle=False, 
                                                target_size=target_size, 
                                                batch_size=1,
                                               class_mode=None)

val_b = ImageDataGenerator().flow_from_directory(img_dir+'/valid',
                                                shuffle=False, 
                                                target_size=target_size, 
                                                batch_size=1,
                                               class_mode=None)

tr_files = tr_b.filenames

val_files = val_b.filenames

tr_data = np.concatenate([tr_b.next() for i in range(tr_b.nb_sample)])

val_data = np.concatenate([val_b.next() for i in range(val_b.nb_sample)])



Found 4092 images belonging to 447 classes.
Found 452 images belonging to 447 classes.


In [32]:
save_array('tr_data_crops_224_224.bc', tr_data)

save_array('tr_filenames_crops.bc', tr_files)

save_array('val_data_224_224_crops.bc', val_data)

save_array('val_filenames_crops.bc', val_files)


In [35]:
tr_classes = tr_b.classes

In [37]:
val_classes= val_b.classes

In [39]:
save_array('tr_classes_crops.bc', tr_classes)
save_array('val_classes_crops.bc', val_classes)



In [2]:
tr_data = load_array('tr_data_crops_224_224.bc')
val_data = load_array('val_data_224_224_crops.bc')
tr_files = load_array('tr_filenames_crops.bc')
val_files = load_array('val_filenames_crops.bc')
tr_classes = load_array('tr_classes_crops.bc')
val_classes = load_array('val_classes_crops.bc')


In [3]:
tr_mean = np.mean(tr_data, axis=(0,2,3), dtype='float32')

In [4]:
tr_mean_r = tr_mean.reshape((3,1,1))

In [5]:
def std(x): 
    x = x-tr_mean.reshape((3,1,1))
    return x

# Create Sample

In [6]:
tr_idx = np.random.permutation(tr_data.shape[0])[:1200]

tr_samp_x = tr_data[tr_idx, :, :, :]

tr_samp_y = tr_classes[tr_idx]


## test if sample size is big enough

In [16]:
def get_lm():
    return Sequential([
            Lambda(std, input_shape=(3,224,224)),
            Flatten(),
            Dense(num_class, activation='softmax')
        ])

In [17]:
model = get_lm()

In [18]:
model.compile(Adam(lr=0.00001), 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [19]:
model.fit(tr_samp_x, tr_samp_y, validation_data=(val_data, val_classes), shuffle=True, nb_epoch=50)

Train on 1200 samples, validate on 452 samples
Epoch 1/50
1200/1200 [==============================] - 3s - loss: 16.0715 - acc: 0.0017 - val_loss: 16.0812 - val_acc: 0.0022
Epoch 2/50
1200/1200 [==============================] - 3s - loss: 16.0375 - acc: 0.0050 - val_loss: 16.0816 - val_acc: 0.0022
Epoch 3/50
1200/1200 [==============================] - 3s - loss: 16.0375 - acc: 0.0050 - val_loss: 16.0816 - val_acc: 0.0022
Epoch 4/50
1200/1200 [==============================] - 3s - loss: 16.0375 - acc: 0.0050 - val_loss: 16.0816 - val_acc: 0.0022
Epoch 5/50
1200/1200 [==============================] - 3s - loss: 16.0375 - acc: 0.0050 - val_loss: 16.0816 - val_acc: 0.0022
Epoch 6/50
 288/1200 [======>.......................] - ETA: 2s - loss: 16.0062 - acc: 0.0069

KeyboardInterrupt: 

In [26]:
def get_model_3():
    return Sequential([
            Lambda(std, input_shape=(3,224,224)),
            ZeroPadding2D((1,1)),
            Conv2D(64, 3,3, activation='relu'),
            MaxPooling2D((2,2)),
            BatchNormalization(),
            Flatten(input_shape=(3,224,224)),
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dense(num_class, activation='softmax')
        ])

In [28]:
del model
model = get_model_3()

In [29]:
model.compile(Adam(lr=0.0001), 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [30]:
model.fit(tr_samp_x, tr_samp_y, validation_data=(val_data, val_classes), shuffle=True, nb_epoch=50)

Train on 1200 samples, validate on 452 samples
Epoch 1/50
1200/1200 [==============================] - 33s - loss: 6.3556 - acc: 0.0050 - val_loss: 10.8429 - val_acc: 0.0022
Epoch 2/50
1200/1200 [==============================] - 33s - loss: 5.4889 - acc: 0.0800 - val_loss: 6.7618 - val_acc: 0.0111
Epoch 3/50
1200/1200 [==============================] - 33s - loss: 4.5624 - acc: 0.3108 - val_loss: 6.1455 - val_acc: 0.0022
Epoch 4/50
1200/1200 [==============================] - 33s - loss: 3.0564 - acc: 0.7417 - val_loss: 6.0854 - val_acc: 0.0111
Epoch 5/50
1200/1200 [==============================] - 33s - loss: 1.4966 - acc: 0.9400 - val_loss: 6.0935 - val_acc: 0.0088
Epoch 6/50
1200/1200 [==============================] - 33s - loss: 0.6195 - acc: 0.9800 - val_loss: 6.0686 - val_acc: 0.0111
Epoch 7/50
1200/1200 [==============================] - 33s - loss: 0.2663 - acc: 0.9942 - val_loss: 6.0868 - val_acc: 0.0111
Epoch 8/50
1200/1200 [==============================] - 33s - loss: 0.

KeyboardInterrupt: 

In [31]:
def get_model_4():
    return Sequential([
            Lambda(std, input_shape=(3,224,224)),
            ZeroPadding2D((1,1)),
            Conv2D(64, 3,3, activation='relu'),
            MaxPooling2D((2,2)),
            BatchNormalization(),
            ZeroPadding2D((1,1)),
            Conv2D(64, 3,3, activation='relu'),
            MaxPooling2D((2,2)),
            BatchNormalization(),
            Flatten(input_shape=(3,224,224)),
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dense(num_class, activation='softmax')
        ])

In [32]:
del model
model = get_model_4()

In [33]:
model.compile(Adam(lr=0.0001), 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [34]:
model.fit(tr_samp_x, tr_samp_y, validation_data=(val_data, val_classes), shuffle=True, nb_epoch=50)

Train on 1200 samples, validate on 452 samples
Epoch 1/50
1200/1200 [==============================] - 42s - loss: 6.3388 - acc: 0.0025 - val_loss: 9.9685 - val_acc: 0.0000e+00
Epoch 2/50
1200/1200 [==============================] - 42s - loss: 5.1902 - acc: 0.1425 - val_loss: 6.7459 - val_acc: 0.0022
Epoch 3/50
1200/1200 [==============================] - 42s - loss: 3.7259 - acc: 0.5508 - val_loss: 6.2105 - val_acc: 0.0111
Epoch 4/50
1200/1200 [==============================] - 42s - loss: 1.6897 - acc: 0.9300 - val_loss: 6.1357 - val_acc: 0.0155
Epoch 5/50
1200/1200 [==============================] - 42s - loss: 0.5838 - acc: 0.9858 - val_loss: 6.0904 - val_acc: 0.0133
Epoch 6/50
1200/1200 [==============================] - 42s - loss: 0.2335 - acc: 0.9917 - val_loss: 6.0727 - val_acc: 0.0199
Epoch 7/50
1200/1200 [==============================] - 42s - loss: 0.1108 - acc: 0.9975 - val_loss: 6.0752 - val_acc: 0.0199
Epoch 8/50
1200/1200 [==============================] - 42s - loss:

KeyboardInterrupt: 

In [35]:
def get_model_5():
    return Sequential([
            Lambda(std, input_shape=(3,224,224)),
            ZeroPadding2D((1,1)),
            Conv2D(64, 3,3, activation='relu'),
            MaxPooling2D((2,2)),
            BatchNormalization(),
            ZeroPadding2D((1,1)),
            Conv2D(64, 3,3, activation='relu'),
            MaxPooling2D((2,2)),
            BatchNormalization(),
            Flatten(input_shape=(3,224,224)),
            Dense(512, activation='relu'),
            BatchNormalization(),
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dense(num_class, activation='softmax')
        ])

In [36]:
del model
model = get_model_5()

In [37]:
model.compile(Adam(lr=0.0001), 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [38]:
model.fit(tr_samp_x, tr_samp_y, validation_data=(val_data, val_classes), shuffle=True, nb_epoch=50)

Train on 1200 samples, validate on 452 samples
Epoch 1/50
1200/1200 [==============================] - 44s - loss: 6.3959 - acc: 0.0067 - val_loss: 11.2280 - val_acc: 0.0066
Epoch 2/50
1200/1200 [==============================] - 44s - loss: 4.8858 - acc: 0.2192 - val_loss: 6.5569 - val_acc: 0.0088
Epoch 3/50
1200/1200 [==============================] - 44s - loss: 3.4671 - acc: 0.6525 - val_loss: 6.1452 - val_acc: 0.0044
Epoch 4/50
1200/1200 [==============================] - 44s - loss: 1.9444 - acc: 0.9183 - val_loss: 6.1208 - val_acc: 0.0066
Epoch 5/50
1200/1200 [==============================] - 44s - loss: 0.9646 - acc: 0.9767 - val_loss: 6.1226 - val_acc: 0.0111
Epoch 6/50
1200/1200 [==============================] - 44s - loss: 0.5018 - acc: 0.9867 - val_loss: 6.1187 - val_acc: 0.0066
Epoch 7/50
1200/1200 [==============================] - 44s - loss: 0.2957 - acc: 0.9950 - val_loss: 6.1191 - val_acc: 0.0111
Epoch 8/50
1200/1200 [==============================] - 44s - loss: 0.

KeyboardInterrupt: 

In [39]:
def get_model_6():
    return Sequential([
            Lambda(std, input_shape=(3,224,224)),
            ZeroPadding2D((1,1)),
            Conv2D(128, 3,3, activation='relu'),
            MaxPooling2D((2,2)),
            BatchNormalization(),
            ZeroPadding2D((1,1)),
            Conv2D(64, 3,3, activation='relu'),
            MaxPooling2D((2,2)),
            BatchNormalization(),
            ZeroPadding2D((1,1)),
            Conv2D(64, 3,3, activation='relu'),
            MaxPooling2D((2,2)),
            BatchNormalization(),
            Flatten(input_shape=(3,224,224)),
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dense(num_class, activation='softmax')
        ])

In [40]:
del model
model = get_model_6()

In [41]:
model.compile(Adam(lr=0.0001), 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [42]:
model.fit(tr_samp_x, tr_samp_y, validation_data=(val_data, val_classes), shuffle=True, nb_epoch=50)

Train on 1200 samples, validate on 452 samples
Epoch 1/50
1200/1200 [==============================] - 74s - loss: 6.3711 - acc: 0.0058 - val_loss: 10.4732 - val_acc: 0.0066
Epoch 2/50
1200/1200 [==============================] - 74s - loss: 4.7326 - acc: 0.2433 - val_loss: 6.3766 - val_acc: 0.0000e+00
Epoch 3/50
1200/1200 [==============================] - 74s - loss: 3.0307 - acc: 0.7192 - val_loss: 6.1664 - val_acc: 0.0133
Epoch 4/50
1200/1200 [==============================] - 74s - loss: 1.3697 - acc: 0.9617 - val_loss: 6.1595 - val_acc: 0.0155
Epoch 5/50
1200/1200 [==============================] - 74s - loss: 0.5734 - acc: 0.9875 - val_loss: 6.1456 - val_acc: 0.0155
Epoch 6/50
1200/1200 [==============================] - 74s - loss: 0.2787 - acc: 0.9933 - val_loss: 6.1461 - val_acc: 0.0199
Epoch 7/50
1184/1200 [============================>.] - ETA: 0s - loss: 0.1563 - acc: 0.9966

KeyboardInterrupt: 

In [80]:
def get_lm_2():
    return Sequential([
            Lambda(std, input_shape=(3,224,224)),
            Flatten(),
            Dense(256, activation='relu'),
            Dense(num_class, activation='softmax')
        ])

In [81]:
del model
model = get_lm_2()

Exception: Input 0 is incompatible with layer dense_20: expected dtype=float32, found dtype=float64

In [76]:
model.compile(Adam(lr=0.001), 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [77]:
model.fit(tr_samp_x, tr_samp_y, validation_data=(val_data, val_classes), shuffle=True, nb_epoch=50)

Train on 1200 samples, validate on 452 samples
Epoch 1/50
1200/1200 [==============================] - 2s - loss: 16.1047 - acc: 8.3333e-04 - val_loss: 16.0468 - val_acc: 0.0044
Epoch 2/50
1200/1200 [==============================] - 2s - loss: 16.1047 - acc: 8.3333e-04 - val_loss: 16.0468 - val_acc: 0.0044
Epoch 3/50
1200/1200 [==============================] - 2s - loss: 16.1047 - acc: 8.3333e-04 - val_loss: 16.0468 - val_acc: 0.0044
Epoch 4/50
1200/1200 [==============================] - 2s - loss: 16.1047 - acc: 8.3333e-04 - val_loss: 16.0468 - val_acc: 0.0044
Epoch 5/50
1200/1200 [==============================] - 2s - loss: 16.1047 - acc: 8.3333e-04 - val_loss: 16.0468 - val_acc: 0.0044
Epoch 6/50
 896/1200 [=====================>........] - ETA: 0s - loss: 16.1001 - acc: 0.0011

KeyboardInterrupt: 

In [16]:
def get_model_train_all():
    vgg = Vgg16BN()
    model = vgg.model
    last_conv = [i for i, v in enumerate(model.layers) if type(v) is Conv2D][-1]
    for layer in model.layers[: last_conv+1]: layer.trainable=False
    model.layers = model.layers[:-1]
    model.add(Dense(num_class, activation='softmax'))
    return model




In [17]:
model = get_model_train_all()

In [20]:
model.compile(Adam(0.001), 'categorical_crossentropy', metrics=['accuracy'])

In [21]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=15)

Epoch 1/15


MemoryError: Error allocating 411041792 bytes of device memory (out of memory).
Apply node that caused the error: GpuDnnPoolGrad{mode='max'}(GpuContiguous.0, GpuContiguous.0, GpuContiguous.0, TensorConstant{(2,) of 2}, TensorConstant{(2,) of 2}, TensorConstant{(2,) of 0})
Toposort index: 847
Inputs types: [CudaNdarrayType(float32, 4D), CudaNdarrayType(float32, 4D), CudaNdarrayType(float32, 4D), TensorType(int64, vector), TensorType(int64, vector), TensorType(int64, vector)]
Inputs shapes: [(64, 128, 112, 112), (64, 128, 56, 56), (64, 128, 56, 56), (2,), (2,), (2,)]
Inputs strides: [(1605632, 12544, 112, 1), (401408, 3136, 56, 1), (401408, 3136, 56, 1), (8,), (8,), (8,)]
Inputs values: ['not shown', 'not shown', 'not shown', array([2, 2]), array([2, 2]), array([0, 0])]
Outputs clients: [[GpuElemwise{Composite{((i0 * i1) + (i0 * i1 * sgn(i2)))}}[(0, 1)](CudaNdarrayConstant{[[[[ 0.5]]]]}, GpuDnnPoolGrad{mode='max'}.0, GpuElemwise{add,no_inplace}.0)]]

HINT: Re-running with most Theano opt

In [22]:
ImageDataGenerator?

In [ ]:
class VGGNoPretrain():
    def __init__(self):
        self.create()
    
    
    def ConvBlock(self, layers, filters):
        model = self.model
        for i in range(layers):
            model.add(ZeroPadding2D((1, 1)))
            model.add(Convolution2D(filters, 3, 3, activation='relu'))
        model.add(MaxPooling2D((2, 2), strides=(2, 2)))


    def FCBlock(self):
        model = self.model
        model.add(Dense(4096, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))


    def create(self, size, include_top):
        model = self.model = Sequential()

        model.add(Lambda(std, input_shape=(3, 224,224)))
        self.ConvBlock(2, 64)
        self.ConvBlock(2, 128)
        self.ConvBlock(3, 256)
        self.ConvBlock(3, 512)
        self.ConvBlock(3, 512)

        model.add(Flatten())
        self.FCBlock()
        self.FCBlock()
        model.add(Dense(1000, activation='softmax'))
